In [11]:
import os 

In [12]:
#os.chdir("../")

In [13]:
pwd

'd:\\CloudDeplyment\\Image-Segmentation-with-AWS'

In [14]:
# entitiy
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_dir: Path
    checkpoint_dir: Path

In [15]:
# COnfiguration Manager

from VGGClassifier.constants import *
from VGGClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:

    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:

        config= self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_dir)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_dir)
            ])
        

        prepare_callback_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_dir= Path(config.tensorboard_dir),
            checkpoint_dir= Path(config.checkpoint_dir)
        )

        return prepare_callback_config
    



        


In [17]:
## Components

import os 
import urllib.request as request
import zipfile as ZipFile
import tensorflow as tf
import time


In [18]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        ckpt_file_path = str(self.config.checkpoint_dir / "model_checkpoint.h5")
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=ckpt_file_path,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [19]:
try:
    config =  ConfigurationManager()

    prepare_callbacks_config=config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e






[2023-08-10 11:08:06,684- INFO]- common - yaml file: config\config.yaml loaded successfully
[2023-08-10 11:08:06,689- INFO]- common - yaml file: params.yaml loaded successfully
[2023-08-10 11:08:06,692- INFO]- common - created directory at: artifacts
[2023-08-10 11:08:06,694- INFO]- common - created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2023-08-10 11:08:06,695- INFO]- common - created directory at: artifacts\prepare_callbacks\tensorboard


AttributeError: 'WindowsPath' object has no attribute 'endswith'